In [180]:
    """
    api_key = 'djvzi2qwau9s1iae317x3sumhi4ovilo'   # Your API key provided by Idealista
    secret = 'dExwCHGjb87b'   # Your secred code provided by Idealista

    api_key = 'z11k3lqpxg3z6zf80628a8745n8sqrdg'   # Your API key provided by Idealista
        secret = 'Fem4xrnUfG8N'   # Your secred code provided by Idealista"""

"\napi_key = 'djvzi2qwau9s1iae317x3sumhi4ovilo'   # Your API key provided by Idealista\nsecret = 'dExwCHGjb87b'   # Your secred code provided by Idealista\n\napi_key = 'z11k3lqpxg3z6zf80628a8745n8sqrdg'   # Your API key provided by Idealista\n    secret = 'Fem4xrnUfG8N'   # Your secred code provided\xa0by\xa0Idealista"

In [35]:
# Import the necessary libraries
import base64
import requests as rq
import json

# Function to obtain the OAuth token
def get_oauth_token():
    '''
    This function will return our personalised token
    '''
    api_key = 'z11k3lqpxg3z6zf80628a8745n8sqrdg'   # Replace with your API Key
    secret = 'Fem4xrnUfG8N'     # Replace with your Secret Key

    message = api_key + ":" + secret   # Combine the API key and the secret to get our personalised message
    auth = "Basic " + base64.b64encode(message.encode("ascii")).decode("ascii")   # Encode the message

    headers_dic = {
        "Authorization": auth,
        "Content-Type": "application/x-www-form-urlencoded;charset=UTF-8"
    }   # Define our headers

    params_dic = {
        "grant_type": "client_credentials",
        "scope": "read"
    }   # Define the request params

    r = rq.post("https://api.idealista.com/oauth/token",
                headers=headers_dic,
                params=params_dic)

    token = json.loads(r.text)['access_token']   # Obtain the personalised token, as a json

    return token

# API search parameters
base_url = 'https://api.idealista.com/3.5/'  # Base search url
country = 'es'  # Search country (es, it, pt)
language = 'es'  # Search language (es, it, pt, en, ca)
max_items = '50'  # Max items per call, the maximum set by Idealista is 50
operation = 'sale'  # Type of operation (sale, rent)
property_type = 'homes'  # Type of property (homes, offices, premises, garages, bedrooms)
order = 'publicationDate'  # Order by publication date
location_id = '0-EU-ES-28' # Idealista location code for Madrid Province
sort = 'desc'  # Sort from most recent to oldest
bankOffer = 'false'  # If the owner is a bank
maxprice = '850000'  # Max price
minprice = '100000'  # Min price

# Define a function to build the search URL
def define_search_url():
    '''
    This function will create our search URL with the modified filters
    '''
    url = (
        base_url +
        country +
        '/search?operation=' + operation +
        '&maxItems=' + max_items +
        '&order=' + order +  # Order by publication date
        '&locationId=' + location_id +  # Instead of center+distance
        '&propertyType=' + property_type +
        '&sort=' + sort +  # Sort in descending order (newest first)
        '&numPage=%s' +  # Pagination
        '&maxPrice=' + maxprice +
        '&minPrice=' + minprice +
        '&language=' + language
    )

    return url

# Example usage
search_url = define_search_url()
print("Search URL:", search_url)


Search URL: https://api.idealista.com/3.5/es/search?operation=sale&maxItems=50&order=publicationDate&locationId=0-EU-ES-28&propertyType=homes&sort=desc&numPage=%s&maxPrice=850000&minPrice=100000&language=es


In [36]:
def search_api(url):
    '''
    This function will use the token and url created previously, and return our search results.
    '''
    token = get_oauth_token()   #  Get the personalised token

    headers = {'Content-Type': 'Content-Type: multipart/form-data;',   # Define the search headers
               'Authorization' : 'Bearer ' + token}

    content = rq.post(url, headers = headers)   # Return the content from the request

    result = json.loads(content.text)   # Transform the result as a json file

    return result

In [ ]:
# Since we need to give pagination to our search and this is our first search, we will set the pagination as 1
pagination = 1 #
first_search_url = url %(pagination)

In [48]:
print(first_search_url)

https://api.idealista.com/3.5/es/search?operation=sale&maxItems=50&order=publicationDate&locationId=0-EU-ES-28&propertyType=homes&sort=desc&numPage=100&maxPrice=850000&minPrice=100000&language=es


In [49]:
# Proceed to do the search with the paginated url
results = search_api(first_search_url)

In [50]:
# First of all, we can extract 50 results/page, but there are more pages, so we have to define how many pages there are.

total_pages = results['totalPages']

In [51]:
print(total_pages)

268


In [52]:
import pandas as pd

In [42]:
def results_to_df(results):
    '''
    This function will save the json results as a dataframe and return the resulting dataframe
    '''
    df = pd.DataFrame.from_dict(results['elementList'])

    return df

In [53]:
def concat_df(df, df_tot):
    '''
    This function will take the main dataframe (df_tot), and concat it with the given individual dataframe,
    returning the main dataframe
    '''
    df_tot= pd.concat([df_tot,df],ignore_index=True)

    return df_tot

In [54]:
# Proceed to save the obtained results as a dataframe
df = results_to_df(results)

In [55]:
# Since we still don't have a main dataframe where we can store all the data, we will create an empty dataframe
df_tot = pd.DataFrame()
df_tot = concat_df(df, df_tot)

In [63]:
url = first_search_url

In [ ]:
print(url)

https://api.idealista.com/3.5/es/search?operation=sale&maxItems=50&order=publicationDate&locationId=0-EU-ES-28&propertyType=homes&sort=desc&numPage=100&maxPrice=850000&minPrice=100000&language=es


: 

In [58]:
print(df_tot.head(1))

  propertyCode                                          thumbnail  \
0    107253989  https://img4.idealista.com/blur/WEB_LISTING/0/...   

  externalReference  numPhotos floor     price  \
0         INVERSION         50     1  460000.0   

                                           priceInfo propertyType operation  \
0  {'price': {'amount': 460000.0, 'currencySuffix...         flat      sale   

   size  ... has3DTour  has360  hasStaging                          highlight  \
0  87.0  ...     False    True       False  {'groupDescription': 'Destacado'}   

  savedAd notes topNewDevelopment topPlus parkingSpace  newDevelopmentFinished  
0      {}    []             False   False          NaN                     NaN  

[1 rows x 43 columns]


In [ ]:
# This will loop over all the pages in the search results. It start from 2, because we've already gotten the first page
for i in range(pagination+1, total_pages):
    url = define_search_url()
    url = url %(i)   # Add the pagination to the url
    results = search_api(url)   # Get the search results
    df = results_to_df(results)   # Save the results as a dataframe
    df_tot = concat_df(df, df_tot)   # Concat the results to the main dataframe

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [61]:
# Once we have all our data, we just need to save it as a csv file, we have created the following function for that:

file_path = r'C:\Users\nicol\OneDrive\Documentos\VSCLocal\Data\TFM\idealista_sale_03_2025_2.csv'

def df_to_csv(df):
    '''
    This function will take a given dataframe and save it as a csv file
    '''
    df = df.reset_index()   # Reset the index in order to organise the records
    df.to_csv(file_path, index=False)   # Save it into a csv

In [62]:
# Run the function and you'll obtain a csv file with all the extracted data
df_to_csv(df_tot)